### Extract Text

In [8]:
import fitz  # PyMuPDF

def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    pages = []
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)
        text = page.get_text().strip()
        pages.append({
            "page_number": page_number + 1,
            "text": text
        })
    doc.close()
    return pages

# Extract and print text
pdf_path = "./metformin1.pdf"
pages = extract_text_from_pdf(pdf_path)

### Extract Entities

In [13]:
import spacy
nlp = spacy.load("en_core_web_sm")


In [14]:
# Function to extract entities from text
def extract_entities(text):
    doc = nlp(text)
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "label": ent.label_
        })
    return entities

In [15]:
all_entities = []

for page in pages:
    entities = extract_entities(page["text"])
    all_entities.append({
        "page_number": page["page_number"],
        "entities": entities
    })
all_entities[0]

{'page_number': 1,
 'entities': [{'text': '2', 'label': 'CARDINAL'},
  {'text': 'Metformin', 'label': 'PERSON'},
  {'text': 'C4H11N5 •', 'label': 'ORG'},
  {'text': '165.63', 'label': 'CARDINAL'},
  {'text': '12.4', 'label': 'CARDINAL'},
  {'text': '1%', 'label': 'PERCENT'},
  {'text': '6.68', 'label': 'CARDINAL'},
  {'text': '500', 'label': 'CARDINAL'},
  {'text': '850', 'label': 'CARDINAL'},
  {'text': '1000', 'label': 'CARDINAL'},
  {'text': '500', 'label': 'CARDINAL'},
  {'text': '850', 'label': 'CARDINAL'},
  {'text': '1000', 'label': 'CARDINAL'},
  {'text': '500', 'label': 'CARDINAL'},
  {'text': '750', 'label': 'CARDINAL'},
  {'text': '500', 'label': 'CARDINAL'},
  {'text': '4079189', 'label': 'CARDINAL'},
  {'text': 'FDA', 'label': 'ORG'},
  {'text': 'https://www.fda.gov/drugsatfda', 'label': 'ORG'}]}

### Building and storing your FAISS index

In [ ]:
# from sentence_transformers import SentenceTransformer
# import faiss
# import numpy as np
# import json
# import pickle

# # 1. Load the embedding model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# # 2. Flatten entities and prepare metadata
# unique_entities = {}
# metadata = []
# embedding_inputs = []

# for item in all_entities:
#     page = item["page_number"]
#     for ent in item["entities"]:
#         text = ent["text"].strip()
#         key = text.lower()

#         if key not in unique_entities:
#             unique_entities[key] = {
#                 "text": text,
#                 "label": ent["label"],
#                 "pages": [page]
#             }
#             embedding_inputs.append(text)
#         else:
#             unique_entities[key]["pages"].append(page)

# # 3. Compute embeddings
# embeddings = model.encode(embedding_inputs, convert_to_numpy=True)

# # 4. Create FAISS index
# dim = embeddings.shape[1]
# index = faiss.IndexFlatL2(dim)
# index.add(embeddings)

# # 5. Save metadata alongside index
# metadata = list(unique_entities.values())

# # Save FAISS index
# faiss.write_index(index, "entity_index.faiss")

# # Save metadata to disk
# with open("entity_metadata.pkl", "wb") as f:
#     pickle.dump(metadata, f)

# print(f"Stored {len(metadata)} unique entities into FAISS.")

Stored 561 unique entities into FAISS.


### Semantic Chunking